In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
# from googletrans import Translator

## Mount to Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

## Import Dataset



In [3]:
#df_1 = pd.read_excel("/content/drive/MyDrive/BT4222/BT4222/Data/dataset_kickstarter-search_2022-09-10_13-27-12-948.xlsx")
#df_2 = pd.read_excel("/content/drive/MyDrive/BT4222/BT4222/Data/kickstarter-final-comments-project.xlsx")
df_1 = pd.read_excel("data/dataset_kickstarter-search_2022-09-10_13-27-12-948.xlsx")
df_2 = pd.read_excel("data/kickstarter-final-comments-project.xlsx")


# get common key
df_1['link'] = df_1['link'].apply(lambda x: str(x)[:str(x).find("?ref")])
df_2['Link-href'] = df_2['Link-href'].apply(lambda x: str(x)[:str(x).find("?ref")])

# try to join by df_1['link'] and df_2['Link-href'] 
df_merged = df_1.merge(df_2, left_on='link', right_on='Link-href')

In [4]:
df_2.columns

Index(['web-scraper-order', 'web-scraper-start-url', 'Title', 'Description',
       'Status', 'Pledged-Amount', 'Funded-percent', 'Time left', 'Sphere',
       'Location', 'Image-src', 'Link', 'Link-href', 'backer-amount', 'video',
       'description_story', 'description_risks', 'rewards', 'updateCount',
       'commentCount'],
      dtype='object')

In [5]:
df_1.columns

Index(['backers_count', 'blurb', 'categoryId', 'categoryName', 'categorySlug',
       'converted_pledged_amount', 'country', 'country_displayable_name',
       'created_at', 'created_at_formatted', 'creatorAvatar', 'creatorId',
       'creatorName', 'creatorUrl', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline', 'description',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_starrable',
       'launched_at', 'launched_at_formatted', 'link', 'locationId',
       'locationName', 'name', 'photo', 'pledged', 'pubDate', 'slug',
       'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'title', 'url', 'usd_exchange_rate', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [6]:
print("APIfy dataset has", len(df_1.columns), "columns", "and", len(df_1), "rows")
print("Webscraper.io dataset has", len(df_2.columns), "columns", "and", len(df_2), "rows")
print("After merge, combined dataframe train has", len(df_merged.columns), "columns and", len(df_merged), 'rows')

APIfy dataset has 45 columns and 2294 rows
Webscraper.io dataset has 20 columns and 2293 rows
After merge, combined dataframe train has 65 columns and 2266 rows


## Train Test Split After Cleaning (70-15-15)
https://www.researchgate.net/post/Removing_a_low_predictive_column_before_or_after_train_test_split#:~:text=Yes%2C%20you%20should%20remove%20the,but%20not%20before%20the%20split.

- Perform encoding for the target then split

- So we will apply the same cleaning to the test data at the end

In [7]:
# get the target variable - success or failure - binary classfication problem
df_merged = df_merged[(df_merged['state'] == "successful") | (df_merged['state'] == "failed")]
print('after remove \n', df_merged['state'].value_counts())
df_merged['state'] = df_merged['state'].apply(lambda x: 0 if x=="failed" else 1)

train, test = train_test_split(df_merged, test_size=0.15, random_state=69, stratify=df_merged['state'])

after remove 
 successful    991
failed        960
Name: state, dtype: int64


In [8]:
print("train size:", len(train))
print("test size", len(test))

train size: 1658
test size 293


In [9]:
train['state']

108     1
281     1
478     1
1674    0
323     1
       ..
565     1
325     1
807     1
376     1
1103    0
Name: state, Length: 1658, dtype: int64

## Intermediate Data Cleaning

In [10]:
#global remove list 
remove_list = []

### 1. Narrow Scope of Data: Limit to Projects in Singapore

In [11]:
train = train[train['country'] == 'SG']
train = train.drop('country', axis=1)

### 3. Remove columns with the same values for all rows





In [12]:
print("Columns with the same values in all of its rows are:")
for i in train: 
  if len(set(train[i])) == 1 or all(pd.isnull(train[i])):
    remove_list.append(i)
    print(i)

## country and location 
# from APIfy: country_displayable_name
# country_displayable_name all same values if all read in English
# all values are thus the same, remove 'country_displayable_name'

remove_list.append('country_displayable_name')
print('country_displayable_name')

Columns with the same values in all of its rows are:
currency
currency_symbol
currency_trailing_code
current_currency
disable_communication
fx_rate
is_starrable
photo
usd_type
country_displayable_name


### 4. Remove columns that have the exact same values or very similar values as another column, and keep unique columns

There will be duplicate or very similar columns because:
- Merged dataset takes data from 2 data sources and they may overlap:
  - For the columns with the exact same values: 
    - Take the more updated column (the column from the more recent dataset (APIfy)
  - For the columns with very similar values:
    - Take the column with more information
    - Take the column whose values can be compared with each other

In [13]:
### name 
# from APIfy: name
# from webscraper.io: Link
# Link is misleading as it is a combination of name and Description 
# keep name as it is more informative 
train[train['Description'] != train['description']][['Description', 'description', 'Link', 'name']]
remove_list.append('Link')

### description 
# from APIfy: 'description', 'blurb'
# from webscraper.io: 'Description'
train[train['blurb'] != train['Description']][['blurb','Description']]
train[train['description'] != train['Description']][['description','Description']]
# blurb is same as Description, but Description can handle more emojis (more informative)
# description is exactly the same as blurb
# keep Description, remove blurb and description
remove_list.extend(['blurb', 'description'])

### pledged amount 
# from APIfy: 'pledged', 'converted_pledged_amount', 'usd_pledged'
# from webscraper.io: 'Pledged-Amount', 'Funded-percent'

# Take the pledged values from webscrapper.io, remove Pledged-Amount and Funded-percent as it is less recent
# pledged vs converted_pledged_amount, converted_pledged_amount is same but removed decimals from pledged
# Keep pledged since it provides more information than converted_pledged_amount (more specific) 
# Keep pledged since it provides more information than usd_pledged (we want local currency) 
# Since the focus is on local currency SGD, remove usd_exchange_rate and static_usd_rate as they are no longer informative
remove_list.extend(['Pledged-Amount', 'converted_pledged_amount',
                    'usd_pledged', 'Funded-percent', 'usd_exchange_rate',
                    'static_usd_rate'])
# Title, title, name
# take the most recent one which is name or title. Keep name
remove_list.extend(['Title', 'title'])

### url
# from APIfy: 'url'
# from webscraper.io: 'Link-href'
train[train['Link-href'] != train['url']][['Link-href','url']]
# Both 'Link-href' and 'url' point to the same webpage. Keep url as it is from the more updated dataset.
remove_list.append('Link-href')

### backers 
# from APIfy: backers_count
# from webscraper.io: backer-amount
train[train['backer-amount'] != train['backers_count']][['backer-amount','backers_count']]
# Keep backers_count as it is from the more updated dataset.
remove_list.append('backer-amount')

## time variables: created_at, launched_at 
# from APIfy: created_at, created_at_formatted, launched_at, launched_at_formatted
# 'created_at' and 'launched_at' column: unix time, converted time already present in 'created_at_formatted' and 'launched_at_formatted'
# Keep 'created_at_formatted' and 'launched_at_formatted' because they have more information 
# and remove 'created_at' and 'launched_at' columns
remove_list.extend(['created_at', 'launched_at'])
# 'pubDate' and 'launched_at_formatted' have exact same values, remove one of the two
remove_list.append('pubDate')

## deadline 
# from APify: deadline
# from webscraper.io: Time left
# keep deadline because it is more recent. Furthermore 'Time left' is dynamic 
remove_list.append('Time left')

### category 
# from APIfy: categoryId, categoryName, categorySlug 
# from websraper.io: Sphare 
train['categoryName'].value_counts().to_frame()
train['categorySlug'].value_counts().to_frame()
train['categoryId'].value_counts().to_frame()
train['Sphere'].value_counts().to_frame()

# the value counts for categoryId and categorySlug are the same 
# Keep categorySlug because it has more information than categoryId which we cannot use to identify the category
# the value counts for categoryName and categorySlug are different - categoryName has more because there are duplicate categories in different languages
# Keep categorySlug as it provides more information that is language neutral
# remove Sphere as there is incomplete information, categorySlug provides full information about category
remove_list.extend(['categoryId', 'categoryName', 'Sphere'])

### location 
# from APIfy: locationId, locationName
# from webscraper: Location
train['locationId'].value_counts()
train['locationName'].value_counts()
# locationId and locationName are the same
# keep locationName as it provides more information and more recent than Location
remove_list.extend(["locationId", "Location"])

### status and State 
# from APIfy: 'state'
# from webscraper.io: 'Status'
# Keep state as it is more recent than Status
remove_list.append('Status')

### creator
# from APIfy: 'creatorId', 'creatorName'
# keep creatorName as it tells us more information about the creator 
train[['creatorId','creatorName','creatorUrl']]
remove_list.append('creatorId')

## slug and id 
# from APIfy: slug, id
# both are identifiers to the projects 
# just choose 1 to keep: keep id
remove_list.append('slug')

### 5. Remove columns that are redundant 

Data is redundant in helping us with our problem statement when:
- The data is metadata 
- The data contains urls that cannot be accessed

In [14]:
## metadata from webscraper.io
# web-scraper-start-url, web-scraper-order
remove_list.extend(['web-scraper-start-url', 'web-scraper-order'])
#state_changed_at 
remove_list.extend(['state_changed_at'])

#unaccessible urls 
#Image-src and creatorAvatar
remove_list.extend(['Image-src', 'creatorAvatar'])

#creator-url, starrable, created_at, is metadata, creator_name
remove_list.extend(['creatorUrl', 'created_at_formatted', 'creatorName', 'url', 'link'])

In [15]:
#drop all at once
train = train.drop(remove_list, axis = 1)
train.columns

Index(['backers_count', 'categorySlug', 'deadline', 'goal', 'id',
       'launched_at_formatted', 'locationName', 'name', 'pledged', 'spotlight',
       'staff_pick', 'state', 'Description', 'video', 'description_story',
       'description_risks', 'rewards', 'updateCount', 'commentCount'],
      dtype='object')

In [16]:
print("APIfy dataset has", len(df_1.columns), "columns", "and", len(df_1), "rows")
print("Webscraper.io dataset has", len(df_2.columns), "columns", "and", len(df_2), "rows")
print("After clean, combined dataframe train has", len(train.columns), "columns and", len(train), 'rows')
train.columns

APIfy dataset has 45 columns and 2294 rows
Webscraper.io dataset has 20 columns and 2293 rows
After clean, combined dataframe train has 19 columns and 1557 rows


Index(['backers_count', 'categorySlug', 'deadline', 'goal', 'id',
       'launched_at_formatted', 'locationName', 'name', 'pledged', 'spotlight',
       'staff_pick', 'state', 'Description', 'video', 'description_story',
       'description_risks', 'rewards', 'updateCount', 'commentCount'],
      dtype='object')

### 6. Rename columns to be more readable and convert columns to their correct formats



In [17]:
train = train.rename(columns={
    "categorySlug":"category",
    "launched_at_formatted": "launched_at",
    "locationName": "location",
    "Description": "description",
    "updateCount": "update_count",
    "commentCount": "faq_count"
})
train = train[['id', 'name', 'description', 
                       'description_story', 'description_risks', 'rewards',
                       'category', 'pledged', 'goal', 'deadline', 'location', 
                       'state','faq_count', 'update_count', 'backers_count',
                       'spotlight', 'staff_pick', 'video', 'launched_at']]

train['spotlight'] = train['spotlight'].astype(int)
train['staff_pick'] = train['staff_pick'].astype(int)
train['deadline'] = pd.to_datetime(train['deadline'], unit='s')  
train['launched_at'] = pd.to_datetime(train['launched_at'], infer_datetime_format=True)
train['rewards'] = train['rewards'].apply(lambda x: ast.literal_eval(x))

In [18]:
train.head()

,id,name,description,description_story,description_risks,rewards,category,pledged,goal,deadline,location,state,faq_count,update_count,backers_count,spotlight,staff_pick,video,launched_at
108,1691565384,Make 100: City maps weaved of cassette tapes,Personalised a map that contains curated music...,Story\n\n\n\n\n\n\n\n\n\n\n\n\n\nYou'll need a...,We thank all of our cassette tape donors who c...,[{'rewards': 'Pledge S$ 90 or more About $65 ...,technology,2138.00,800,2022-02-02 12:00:05,"Singapore, Singapore",1,4,5.0,17,1,0,https://v2.kickstarter.com/1662722892-eVf1EU6P...,2022-01-08 12:00:05+00:00
281,320949924,Retrograde Hard Enamel Pins,A series of Enamel Pins based on the combinati...,StoryA series of pins based on a set of illust...,- The possibility of minor changes in details ...,[{'rewards': 'Pledge S$ 10 or more About US$ ...,design/graphic design,10476.00,1200,2021-03-16 12:50:23,"Singapore, Singapore",1,12,9.0,226,1,0,NaN,2021-02-18 13:50:23+00:00
478,1956852023,Owl-Carina: The Sound of Wings | MAKE 100,Cute handmade owl ocarinas and whistles. Potte...,Story\n\n\nOwl-carina\n\n\n\n\nWhat can these ...,Shipping and delivery might break the pieces. ...,[{'rewards': 'Pledge S$ 2 or more About $2 ...,crafts/pottery,7316.00,100,2020-02-24 17:46:24,"Singapore, Singapore",1,11,9.0,152,1,1,NaN,2020-01-25 17:46:24+00:00
1674,1565012664,Alohomora - Magical Automated Safety Gate Unlo...,"Alohomora, an automated unlocking module for b...",NaN,NaN,[{'rewards': 'Pledge S$ 2 or more About $2 ...,technology/gadgets,2437.00,3000,2017-12-15 12:02:03,"Singapore, Singapore",0,0,4.0,112,0,0,https://v2.kickstarter.com/1662722793-jJXI4ird...,2017-11-15 12:02:03+00:00
323,1255089705,Vario WW1 1918 Trench Watch,Vario's 3rd watch collection inspired by WW1 p...,StoryAll watches will come with a strap and ba...,We've had numerous successful crowdfunding cam...,[{'rewards': 'Pledge S$ 20 or more About US$ ...,design/product design,297149.03,30000,2020-11-25 04:00:00,"Singapore, Singapore",1,631,31.0,741,1,0,https://v2.kickstarter.com/1662645846-J7scf2mw...,2020-11-11 02:59:03+00:00


In [19]:
train.columns

Index(['id', 'name', 'description', 'description_story', 'description_risks',
       'rewards', 'category', 'pledged', 'goal', 'deadline', 'location',
       'state', 'faq_count', 'update_count', 'backers_count', 'spotlight',
       'staff_pick', 'video', 'launched_at'],
      dtype='object')

In [20]:
len(train)
len(train.columns)

19

## Apply the same data cleaning to test set

In [21]:
test = test.drop(remove_list, axis = 1)

test = test.rename(columns={
    "categorySlug":"category",
    "launched_at_formatted": "launched_at",
    "locationName": "location",
    "Description": "description",
    "updateCount": "update_count",
    "commentCount": "faq_count"
})
test = test[['id', 'name', 'description', 
                       'description_story', 'description_risks', 'rewards',
                       'category', 'pledged', 'goal', 'deadline', 'location', 
                       'state','faq_count', 'update_count', 'backers_count',
                       'spotlight', 'staff_pick', 'video', 'launched_at']]

test['spotlight'] = test['spotlight'].astype(int)
test['staff_pick'] = test['staff_pick'].astype(int)
test['deadline'] = pd.to_datetime(test['deadline'], unit='s')  
test['launched_at'] = pd.to_datetime(test['launched_at'], infer_datetime_format=True)
test['rewards'] = test['rewards'].apply(lambda x: ast.literal_eval(x))

## Export to excel

In [22]:
train.to_csv("data/kickstarter_train.csv", index=False)
test.to_csv("data/kickstarter_test.csv", index=False)